## Building a GPT

In [69]:
# read it in to inspect it
with open('Mahabharat.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [70]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  14839497


In [76]:
# let's look at the first 1000 characters
print(text[:1000])

The Complete Mahabharata in English
The Mahabharata
of
Krishna-Dwaipayana Vyasa

BOOK 1
ADI PARVA
Translated into English Prose from the Original Sanskrit Text by Kisari Mohan Ganguli [1883-1896]
Scanned at sacred-texts.com, 2003. Proofed at Distributed Proofing, Juliet Sutherland, Project Manager. Additional proofing
and formatting at sacred-texts.com, by J. B. Hare.
TRANSLATOR'S PREFACE
The object of a translator should ever be to hold the mirror upto his author. That being so, his chief duty is to represent so far as
practicable the manner in which his author's ideas have been expressed, retaining if possible at the sacrifice of idiom and taste
all the peculiarities of his author's imagery and of language as well. In regard to translations from the Sanskrit, nothing is easier
than to dish up Hindu ideas, so as to make them agreeable to English taste. But the endeavour of the present translator has been
to give in the following pages as literal a rendering as possible of the great wo

In [77]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#&'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz—
84


In [78]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[64, 65, 65, 2, 76, 64, 61, 74, 61]
hii there


In [79]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([14839497]) torch.int64
tensor([45, 64, 61,  2, 28, 71, 69, 72, 68, 61, 76, 61,  2, 38, 57, 64, 57, 58,
        64, 57, 74, 57, 76, 57,  2, 65, 70,  2, 30, 70, 63, 68, 65, 75, 64,  0,
        45, 64, 61,  2, 38, 57, 64, 57, 58, 64, 57, 74, 57, 76, 57,  0, 71, 62,
         0, 36, 74, 65, 75, 64, 70, 57, 11, 29, 79, 57, 65, 72, 57, 81, 57, 70,
        57,  2, 47, 81, 57, 75, 57,  0,  0, 27, 40, 40, 36,  2, 14,  0, 26, 29,
        34,  2, 41, 26, 43, 47, 26,  0, 45, 74, 57, 70, 75, 68, 57, 76, 61, 60,
         2, 65, 70, 76, 71,  2, 30, 70, 63, 68, 65, 75, 64,  2, 41, 74, 71, 75,
        61,  2, 62, 74, 71, 69,  2, 76, 64, 61,  2, 40, 74, 65, 63, 65, 70, 57,
        68,  2, 44, 57, 70, 75, 67, 74, 65, 76,  2, 45, 61, 80, 76,  2, 58, 81,
         2, 36, 65, 75, 57, 74, 65,  2, 38, 71, 64, 57, 70,  2, 32, 57, 70, 63,
        77, 68, 65,  2, 52, 14, 21, 21, 16, 11, 14, 21, 22, 19, 54,  0, 44, 59,
        57, 70, 70, 61, 60,  2, 57, 76,  2, 75, 57, 59, 74, 61, 60, 11, 76, 61,
     

In [82]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [83]:
block_size = 8
train_data[:block_size+1]

tensor([45, 64, 61,  2, 28, 71, 69, 72, 68])

In [84]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([45]) the target: 64
when input is tensor([45, 64]) the target: 61
when input is tensor([45, 64, 61]) the target: 2
when input is tensor([45, 64, 61,  2]) the target: 28
when input is tensor([45, 64, 61,  2, 28]) the target: 71
when input is tensor([45, 64, 61,  2, 28, 71]) the target: 69
when input is tensor([45, 64, 61,  2, 28, 71, 69]) the target: 72
when input is tensor([45, 64, 61,  2, 28, 71, 69, 72]) the target: 68


In [86]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[61, 70, 76,  2, 70, 65, 63, 64],
        [76,  2, 43, 57, 69, 57,  2, 79],
        [ 2, 62, 71, 74, 76, 64,  2, 62],
        [ 2, 76, 64, 61,  2, 59, 71, 77]])
targets:
torch.Size([4, 8])
tensor([[70, 76,  2, 70, 65, 63, 64, 76],
        [ 2, 43, 57, 69, 57,  2, 79, 65],
        [62, 71, 74, 76, 64,  2, 62, 65],
        [76, 64, 61,  2, 59, 71, 77, 74]])
----
when input is [61] the target: 70
when input is [61, 70] the target: 76
when input is [61, 70, 76] the target: 2
when input is [61, 70, 76, 2] the target: 70
when input is [61, 70, 76, 2, 70] the target: 65
when input is [61, 70, 76, 2, 70, 65] the target: 63
when input is [61, 70, 76, 2, 70, 65, 63] the target: 64
when input is [61, 70, 76, 2, 70, 65, 63, 64] the target: 76
when input is [76] the target: 2
when input is [76, 2] the target: 43
when input is [76, 2, 43] the target: 57
when input is [76, 2, 43, 57] the target: 69
when input is [76, 2, 43, 57, 69] the target: 57
when input is [76, 

In [87]:
print(xb) # our input to the transformer

tensor([[61, 70, 76,  2, 70, 65, 63, 64],
        [76,  2, 43, 57, 69, 57,  2, 79],
        [ 2, 62, 71, 74, 76, 64,  2, 62],
        [ 2, 76, 64, 61,  2, 59, 71, 77]])


In [88]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 84])
tensor(4.9242, grad_fn=<NllLossBackward0>)

"A&"#Cr59cIE"b
QHHKc\tMMtJ
E4NAZ0RBNKZr.DU'Z!iyQ]ERSunceD jPu[2jJd9J#eEMtZSI1 35RU\KiylZ3__7OV5VAE`U


In [89]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [90]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.823068618774414


In [91]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


37K?a42#EJ2!
]S:Hdn1yOa`CVQ"\t-e4yw-!GD47_c9YZ)xU"3VQCt5—J
4?Y5dhU
z;y2uueA;rUO0mAElX]_,gWlvn`:3gW24MDhVJ!oHJvkJ8fD#P0Hcm!0RlRR'u`lNiIRka 5!'pv
YdQB
8zQt(H)gOqwjW]b8f(zIWzwv]eN&)w1urqwf''7U]"DEdP&.TV!ooIQ\Lgru2"5E
(;
'D.pw)3B!SqwIF:XPL:MVMlFLYw-6LID4J1[z
:k?Z4LYT7UVnCxQj"4l_lzxDH—OXMRN(.Jk'u!n8av!02?EgF65t48.aRXl—g#?h!8jWvnEZ0Q AIWC1)gDEg)__R_B`CZTT:j]i;mC)vifM;KbNK37QpGyMjTfUH0nj)wq&"v_NLJO,KMa&C6wA?q,.4
_83rB 4V
fvKegYsS`"S.pwJO5ucEvqbxFnu4)w,FgSAUo I(3:Pr2a0nz3iEs1x2JKNmyNK9—?YOoqcfmV"s


## The mathematical trick in self-attention

In [92]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [94]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [95]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [96]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [97]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [98]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [99]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [100]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [101]:
k.var()

tensor(1.0449)

In [102]:
q.var()

tensor(1.0700)

In [103]:
wei.var()

tensor(1.0918)

In [104]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [105]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [106]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [107]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [108]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [109]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('Mahabharat.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.21218 M parameters
step 0: train loss 4.6268, val loss 4.6188
step 100: train loss 2.6066, val loss 2.6264
step 200: train loss 2.4926, val loss 2.5111
step 300: train loss 2.4358, val loss 2.4324
step 400: train loss 2.3582, val loss 2.3730
step 500: train loss 2.2769, val loss 2.2963
step 600: train loss 2.2068, val loss 2.2292
step 700: train loss 2.1536, val loss 2.1802
step 800: train loss 2.1014, val loss 2.1197
step 900: train loss 2.0650, val loss 2.0848
step 1000: train loss 2.0144, val loss 2.0326
step 1100: train loss 1.9842, val loss 2.0027
step 1200: train loss 1.9501, val loss 1.9749
step 1300: train loss 1.9291, val loss 1.9476
step 1400: train loss 1.8868, val loss 1.9136
step 1500: train loss 1.8704, val loss 1.8870
step 1600: train loss 1.8433, val loss 1.8596
step 1700: train loss 1.8258, val loss 1.8494
step 1800: train loss 1.8037, val loss 1.8259
step 1900: train loss 1.7876, val loss 1.8049
step 2000: train loss 1.7752, val loss 1.7839
step 2100: train loss 1.7

In [117]:

# generate from the model
#context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=20000)[0].tolist()))


from
company victor, the are the celestice to commentanumed. And the protected, all the eld
of Jigijita), O king. After however, O antire hopession armysy, those him sacrificial colliwed a held noth to thou elephants at Taming lone to forer poince, we the votants of Duryodhana, abdurent ingnum. And exclouding, from thou is their
however wheeth And cosequea my men while not, and with not shafts Bhasara saying
infin. Krunga taksa, haveous, apppen the Benal
Prithatya feed thee high-seek of marile be keepen their chards word Dauspaded Drounting them into
marissed in this being dimer to mementy. Thou diversant as thee from that chevereupon to the Pandava not univing any jepty, the words, plowess.
It the fruit of counted wounding pam. He Nara is the all sup all holebjacted the bodyied on in stendanth ambapayething not for with the Jarnas arsond, with their perform their spereed to yet creanses
of the person of Pa and Seana gorded by esire a
palicial and hygit, soul, O fill do depation of th